# Umair Ahmad
# 21i-2081
## Umair.Ahmad@Nu.edu.pk

# Natural Language Processing Assignment #5
# Fake News Detection 

In [69]:
from os import listdir
import numpy as np
import pandas as pd
import spacy
from urduhack.preprocessing import normalize_whitespace
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import math

# Data Preprocessing 

In [70]:
def data_cleaning(data,stop_words):
    data= normalize_whitespace(data)
    data = data.replace('۔',' ')
    data = data.replace('،',' ')
    data = data.replace('\n',' ')
    data=data.replace('(',' ')
    data=data.replace(')',' ')
    nlp = spacy.blank('ur')
    doc = nlp(data)
    cleaned_data =[]
    for x in doc:
        if str(x) not in stop_words:
            cleaned_data.append(str(x))
    return cleaned_data

Stop_Words=open('C:/Users/Umair/Downloads/NLP assignmnent 5/stopwords-ur.txt', encoding='utf-8').read()
Stop_Words=Stop_Words.replace('\n',' ')

#  Features Extraction
- Function "get_data" recive the three arguments Path of for filereading
-  Pass list of Stop word if you want to remove stopwords otherwise pass the empty string
-  Set BN to True if you want to remove the duplicates from each features otherwise set to False

In [71]:
def get_data(path,stop_words,BN):
    sentences=[]
    if("." not in path):
        dir_names=listdir(path)
        for file in dir_names:
            doc=open(str(path)+"/"+str(file), encoding='utf-8').read()
            lst=data_cleaning(doc,stop_words)
            if BN == True:
                lst = list(set(lst))
            sentences=sentences+lst
    else:
        doc=open(path, encoding='utf-8').read()
        lst=data_cleaning(doc,stop_words)
        if BN == True:
            lst = list(set(lst))
        sentences=sentences+lst
    return sentences

# Feature Preparations for Multinomial NB With Stopwords removal

In [72]:
MN_Fake_Data=get_data('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Fake',Stop_Words,False)
MN_Real_Data=get_data('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Real',Stop_Words,False)
print(len(MN_Fake_Data))
print(len(MN_Real_Data))

49261
74100


# Feature Preparations for Boolean NB With Stopwords removal

In [73]:
BN_Fake_Data=get_data('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Fake',Stop_Words,True)
BN_Real_Data=get_data('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Real',Stop_Words,True)
print(len(BN_Fake_Data))
print(len(BN_Real_Data))

33036
46513


# Feature Preparations for MultiNomial NB Without Stopwords removal

In [74]:
MN_Fake_Data_with_Stopwords=get_data('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Fake',"",False)
MN_Real_Data_with_Stopwords=get_data('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Real',"",False)
print(len(MN_Fake_Data_with_Stopwords))
print(len(MN_Real_Data_with_Stopwords))

87510
131014


# Feature Preparations for Boolean NB Without Stopwords removal

In [75]:
BN_Fake_Data_with_Stopwords=get_data('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Fake',"",True)
BN_Real_Data_with_Stopwords=get_data('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Real',"",True)
print(len(BN_Fake_Data_with_Stopwords))
print(len(BN_Real_Data_with_Stopwords))

44431
61128


# Prior Calculation for Fake and Real Class

In [76]:
Fake_train_dir=listdir('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Fake')
Real_train_dir=listdir('C:/Users/Umair/Downloads/NLP assignmnent 5/Train/Real')
Prior_of_Train_Fake=len(Fake_train_dir)/(len(Fake_train_dir)+len(Real_train_dir))
Prior_of_Train_Real=len(Real_train_dir)/(len(Fake_train_dir)+len(Real_train_dir))
print(Prior_of_Train_Fake)
print(Prior_of_Train_Real)

0.45141065830721006
0.54858934169279


# Traning of Features
- "Traning" function receive the two arguments Fake Data and Real Data 
- Returns the Tranined Conditional Probabilities of the given data 

In [77]:
def Traning(Fake_data,Real_data):
    wordsFreq_fake = defaultdict(float)
    wordsFreq_real = defaultdict(float)
    cond_prob_fake=defaultdict(float)
    cond_prob_real=defaultdict(float)
    Vocab=defaultdict(float)
    for word in Fake_data:
        wordsFreq_fake[str(word)]+=1
        Vocab[str(word)]+=1
    for word in Real_data:
        wordsFreq_real[str(word)]+=1
        Vocab[str(word)]+=1
    for key,value in Vocab.items():
        cond_prob_fake[str(key)]=(wordsFreq_fake[str(key)]+1)/(len(Fake_data)+len(Vocab))
        cond_prob_real[str(key)]=(wordsFreq_real[str(key)]+1)/(len(Real_data)+len(Vocab))
    return cond_prob_real,cond_prob_fake

MN_cond_prob_real,MN_cond_prob_fake=Traning(MN_Fake_Data,MN_Real_Data)
BN_cond_prob_real,BN_cond_prob_fake=Traning(BN_Fake_Data,BN_Real_Data)
MN_cond_prob_real_SW,MN_cond_prob_fake_SW=Traning(MN_Fake_Data_with_Stopwords,MN_Real_Data_with_Stopwords)
BN_cond_prob_real_SW,BN_cond_prob_fake_SW=Traning(BN_Fake_Data_with_Stopwords,BN_Real_Data_with_Stopwords)

# Visualization of Sorted Multinomial NB Real Conditional Probabilties

In [78]:
uni_gram_freq=pd.DataFrame(MN_cond_prob_real.items(), columns=['Uni_Grams', 'Freq'])
uni_gram_freq=uni_gram_freq.sort_values('Freq', ascending=False)
uni_gram_freq.to_csv(r'MN_cond_prob_real.csv')
uni_gram_freq  

,Uni_Grams,Freq
61,نہیں,0.006826
17,پاکستان,0.005527
136,کہا,0.004228
6,بعد,0.004000
252,ساتھ,0.003544
...,...,...
7974,وئير,0.000011
5558,تيرہ,0.000011
7977,ڈيبٹ,0.000011
5555,يونیورس,0.000011


## Visualization of Sorted Multinomial NB Fake Conditional Probabilties

In [79]:
uni_gram_freq=pd.DataFrame(MN_cond_prob_fake.items(), columns=['Uni_Grams', 'Freq'])
uni_gram_freq=uni_gram_freq.sort_values('Freq', ascending=False)
uni_gram_freq.to_csv(r'MN_cond_prob_fake.csv')
uni_gram_freq

,Uni_Grams,Freq
17,پاکستان,0.006882
61,نہیں,0.006088
252,ساتھ,0.004148
102,مطابق,0.003894
41,ہونے,0.003751
...,...,...
10561,محال,0.000016
10560,جینا,0.000016
10559,پڑجاتی,0.000016
10558,ایئرکنڈیشنرکی,0.000016


# Visualization of Sorted Boolean NB Real Conditional Probabilties

In [80]:
uni_gram_freq=pd.DataFrame(BN_cond_prob_real.items(), columns=['Uni_Grams', 'Freq'])
uni_gram_freq=uni_gram_freq.sort_values('Freq', ascending=False)
uni_gram_freq.to_csv(r'BN_cond_prob_real.csv')
uni_gram_freq

,Uni_Grams,Freq
52,نہیں,0.003507
120,بعد,0.003108
57,مطابق,0.002742
15,کہا,0.002742
92,ہونے,0.002576
...,...,...
7735,پرل,0.000017
7736,8ماڈلز,0.000017
7737,7روپے,0.000017
7738,ہيڈ,0.000017


# Visualization of Sorted Boolean NB Fake Conditional Probabilties

In [81]:
uni_gram_freq=pd.DataFrame(BN_cond_prob_fake.items(), columns=['Uni_Grams', 'Freq'])
uni_gram_freq=uni_gram_freq.sort_values('Freq', ascending=False)
uni_gram_freq.to_csv(r'BN_cond_prob_fake.csv')
uni_gram_freq

,Uni_Grams,Freq
52,نہیں,0.003598
57,مطابق,0.003256
92,ہونے,0.003127
120,بعد,0.002977
473,دیا,0.002784
...,...,...
10561,ناقابلِ,0.000021
10560,سینٹرہرجگہ,0.000021
10559,ایئرکنڈیشنرکو,0.000021
10558,محال,0.000021


# Testing of TestData
- Function "Testing" receive the Conditional probabilties and Prior of the both classes 
- Return the Predicited Label from log probabilties 

In [82]:
def Testing(cond_prob_real,cond_prob_fake,Prior_of_Fake,Prior_of_Real,text):
    score={"Real":0,"Fake":0}
    score["Real"]=math.log(Prior_of_Real)
    score["Fake"]=math.log(Prior_of_Fake)
    for x in text:
        if cond_prob_real[str(x)]>0:
            score["Real"]+=math.log(cond_prob_real[str(x)])
        if cond_prob_fake[str(x)]>0:
            score["Fake"]+=math.log(cond_prob_fake[str(x)])
    if score["Real"]>score["Fake"]:
        return "Real"
    else:
        return "Fake"

# Getting Labels of all the Test Data

In [83]:
Fake_path='C:/Users/Umair/Downloads/NLP assignmnent 5/Test/Fake'
Real_path='C:/Users/Umair/Downloads/NLP assignmnent 5/Test/Real'
Fake_dir_names=listdir(Fake_path)
Real_dir_names=listdir(Real_path)

def cal_labels(real_cond,fake_cond,fake_prior,real_prior,stop_words,BN):
    Ypre=[]
    for x in Fake_dir_names:
        text=get_data(Fake_path+"/"+x,stop_words,BN)
        ypred=Testing(real_cond,fake_cond,fake_prior,real_prior,text)
        Ypre.append(ypred) 
    for x in Real_dir_names:
        text=get_data(Real_path+"/"+x,stop_words,BN)
        ypred=Testing(real_cond,fake_cond,fake_prior,real_prior,text)
        Ypre.append(ypred)
    return Ypre

MN_Ypre=cal_labels(MN_cond_prob_real,MN_cond_prob_fake,Prior_of_Train_Fake,Prior_of_Train_Real,Stop_Words,False)
BN_Ypre=cal_labels(BN_cond_prob_real,BN_cond_prob_fake,Prior_of_Train_Fake,Prior_of_Train_Real,Stop_Words,True)
MN_Ypre_SW=cal_labels(MN_cond_prob_real_SW,MN_cond_prob_fake_SW,Prior_of_Train_Fake,Prior_of_Train_Real,"",False)
BN_Ypre_SW=cal_labels(BN_cond_prob_real_SW,BN_cond_prob_fake_SW,Prior_of_Train_Fake,Prior_of_Train_Real,"",True)

# Report

## Result Anaylsis
After seeing the results of 
- Na¨ıve Bayes classifier
- Boolean Na¨ıve Bayes classifier
- Experiment with/without using stop words

Metrics for evaluating performance of this classifier are precision,recall, f1-score and accuracy.
After Traning of a Na¨ıve Bayes classifier on the data set provided on all four combination mentioned above.
These trained model is tested on the testset of Fake and Real new.

- Train Dataset(Real=350,Fake=288)
- Test Dataset (Real=150,Fake=262)

All the classification report are shown below also the average accuracy score of all the four trained models are

- **Accuracy Score of Multinomial NB With    Stop Words:    0.70**
- **Accuracy Score of Boolean     NB With    Stop Words:    0.78**
- **Accuracy Score of Multinomial NB Without Stop Words:    0.73**
- **Accuracy Score of Boolean     NB Without Stop Words:    0.79**

Rest of the Evaluation measures Precision,Recall and F1 measures are shown below

## Conculsion 
We can achive good results by choosing Boolean Naive Bayes without stopwords

# Classification Report of Accuracy, Precision, Recall and F1 Measures.

In [84]:
y_Real = ["Real"] * len(Real_dir_names)
y_Fake = ["Fake"] * len(Fake_dir_names)
Ytest=y_Fake+y_Real
MN_acc=accuracy_score(Ytest, MN_Ypre)
BN_acc=accuracy_score(Ytest, BN_Ypre)
MN_acc_SW=accuracy_score(Ytest, MN_Ypre_SW)
BN_acc_SW=accuracy_score(Ytest, BN_Ypre_SW)
print("Accuracy Score of Multinomial NB With    Stop Words:    "+str(MN_acc))
print("Accuracy Score of Boolean     NB With    Stop Words:    "+str(BN_acc))
print("Accuracy Score of Multinomial NB Without Stop Words:    "+str(MN_acc_SW))
print("Accuracy Score of Boolean     NB Without Stop Words:    "+str(BN_acc_SW))

Accuracy Score of Multinomial NB With    Stop Words:    0.6946564885496184
Accuracy Score of Boolean     NB With    Stop Words:    0.7748091603053435
Accuracy Score of Multinomial NB Without Stop Words:    0.7290076335877863
Accuracy Score of Boolean     NB Without Stop Words:    0.7824427480916031


# Classification Report of Multinomial NB With Stop Words

In [85]:
target_names = ["Fake", "Real"]
print(classification_report(Ytest, MN_Ypre, target_names=target_names))

              precision    recall  f1-score   support

        Fake       0.65      0.62      0.63       112
        Real       0.72      0.75      0.74       150

    accuracy                           0.69       262
   macro avg       0.69      0.68      0.69       262
weighted avg       0.69      0.69      0.69       262



# Classification Report of Boolean NB With Stop Words

In [86]:
print(classification_report(Ytest, BN_Ypre, target_names=target_names))

              precision    recall  f1-score   support

        Fake       0.79      0.64      0.71       112
        Real       0.77      0.87      0.82       150

    accuracy                           0.77       262
   macro avg       0.78      0.76      0.76       262
weighted avg       0.78      0.77      0.77       262



# Classification Report of Multinomial NB Without Stop Words

In [87]:
print(classification_report(Ytest, MN_Ypre_SW, target_names=target_names))

              precision    recall  f1-score   support

        Fake       0.70      0.64      0.67       112
        Real       0.75      0.79      0.77       150

    accuracy                           0.73       262
   macro avg       0.72      0.72      0.72       262
weighted avg       0.73      0.73      0.73       262



# Classification Report of Boolean NB Without Stop Words

In [88]:
print(classification_report(Ytest, BN_Ypre_SW, target_names=target_names))

              precision    recall  f1-score   support

        Fake       0.81      0.64      0.72       112
        Real       0.77      0.89      0.82       150

    accuracy                           0.78       262
   macro avg       0.79      0.76      0.77       262
weighted avg       0.79      0.78      0.78       262

